## Lesson 12 - Administrative and perform a backup (Enterprise Edition)





### Table of Contents

* [Introduction of Neo4j backup](#intro_neo4j_backup)
* [Administrative commands](#administrative_commands)
* [Online and offline backups](#online_offline_backup)
* [Dump and load databases](#dump_and_load_databases)
* [How to perform an online backup of a Neo4j database](#neo4j-admin)
* [Understanding memory configurations for neo4j-admin backup](#memory_heap_size)
* [Moving Neo4j Desktop database location](#move_db_location)





<a id="intro_neo4j_backup"></a>
## Introduction
Designing an appropriate backup strategy for your Neo4j DBMS is a fundamental part of operations. The backup strategy should take into account elements such as:

- Demands on performance during backup actions.
- Tolerance for data loss in case of failure.
- Tolerance for downtime in case of failure.
- Data volumes.

The backup strategy will answer question such as:

- What type of backup method used; online or offline backups?
- What physical setup meets our demands?
- What backup media — offline or remote storage, cloud storage etc. — should we use?
- How long do we archive backups for?
- With what frequency should we perform backups;

If using online backups:
- How often should we perform full backups?
- How often should we perform incremental backups?
- How do we test recovery routines, and how often?

<a id="administrative_commands"></a>
## Neo4j Administrative Commands

Before using administrative commands, it is important to understand the difference between stopped databases, and dropped databases:

Databases that are stopped with the STOP command are completely shutdown, and may be started again through the START command. In a Causal Cluster, as long as a database is in a shutdown state, it can not be considered available to other members of the cluster. It is not possible to do online backups against shutdown databases and they need to be taken into special consideration during disaster recovery, as they do not have a running Raft machine while shutdown.

Dropped databases are completely removed and are not intended to be used again at all.

The following Cypher commands are used on the system database to manage multiple databases:

| Command     | Description                                                                      |
|:-----------:|:--------------------------------------------------------------------------------:|
| CREATE DATABASE name   | Create and start a new database.                                      |
| DROP DATABASE name     | Drop (remove) an existing database.                                   |
| START DATABASE name    | Start a database that has been stopped.                               |
| STOP DATABASE name     | Shut down a database.                                                 |
| SHOW DATABASE name     | Show the status of a specific database.                               |
| SHOW DATABASES         | Show the name and status of all the databases.                        |
| SHOW DEFAULT DATABASE  | Show the name and status of the default database.                     |

Naming rules for databases are as follows:

- Length must be between 3 and 63 characters.
- The first character of a name must be an ASCII alphabetic character.
- Subsequent characters must be ASCII alphabetic or numeric characters, dots or dashes; [a..z][0..9].-
- Names are case-insensitive and normalized to lowercase.
- Names that begin with an underscore and with the prefix system are reserved for internal use.

### Show the status of a specific database
```
neo4j@system> SHOW DATABASE neo4j;
```

In standalone mode:

```
+-----------------------------------------------------------------------------------------------+
| name    | address          | role         | requestedStatus | currentStatus | error | default |
+-----------------------------------------------------------------------------------------------+
| "neo4j" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
+-----------------------------------------------------------------------------------------------+

1 row available after 100 ms, consumed after another 6 ms
```

Or in a Causal Cluster:

```
+-----------------------------------------------------------------------------------------------+
| name    | address          | role       | requestedStatus | currentStatus | error | default   |
+-----------------------------------------------------------------------------------------------+
| "neo4j" | "localhost:7687" | "leader"   | "online"        | "online"      | ""    | TRUE      |
| "neo4j" | "localhost:7688" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "neo4j" | "localhost:7689" | "follower" | "online"        | "online"      | ""    | TRUE      |
+-----------------------------------------------------------------------------------------------+

3 row available after 100 ms, consumed after another 6 ms
```

### Show the status of all databases

```
neo4j@system> SHOW DATABASES;
```

In standalone mode:

```
+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

2 rows available after 5 ms, consumed after another 1 ms
```

Or in a Causal Cluster:

```
+------------------------------------------------------------------------------------------------+
| name     | address          | role       | requestedStatus | currentStatus | error | default   |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "leader"   | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7688" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7689" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "system" | "localhost:7687" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7688" | "leader"   | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7689" | "follower" | "online"        | "online"      | ""    | FALSE     |
+------------------------------------------------------------------------------------------------+

6 rows available after 5 ms, consumed after another 1 ms
```

Switching between online and offline states is achieved using the START DATABASE and STOP DATABASE commands.

### Show the status of the default database

```
neo4j@system> SHOW DEFAULT DATABASE;
```

In standalone mode:

```
+-------------------------------------------------------------------------------------+
| name    | address          | role         | requestedStatus | currentStatus | error |
+-------------------------------------------------------------------------------------+
| "neo4j" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    |
+-------------------------------------------------------------------------------------+

1 row available after 57 ms, consumed after another 2 ms
```

Or in a Causal Cluster:
```
+-------------------------------------------------------------------------------------+
| name    | address          | role       | requestedStatus | currentStatus | error   |
+-------------------------------------------------------------------------------------+
| "neo4j" | "localhost:7687" | "follower" | "online"        | "online"      | ""      |
| "neo4j" | "localhost:7688" | "leader"   | "online"        | "online"      | ""      |
| "neo4j" | "localhost:7689" | "follower" | "online"        | "online"      | ""      |
+-------------------------------------------------------------------------------------+

3 row available after 57 ms, consumed after another 2 ms
```
You can change the default database by using dbms.default_database, and restarting the server.

### Create a database
```
neo4j@system> CREATE DATABASE sales;
```
```
0 rows available after 108 ms, consumed after another 0 ms
```

```
neo4j@system> SHOW DATABASES;
```

In standalone mode:

```
+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
| "sales"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

3 rows available after 4 ms, consumed after another 1 ms
```

Or in a Causal Cluster:
```
+------------------------------------------------------------------------------------------------+
| name     | address          | role       | requestedStatus | currentStatus | error | default   |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "leader"   | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7688" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7689" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "system" | "localhost:7687" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7688" | "leader"   | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7689" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7687" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7688" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7689" | "leader"   | "online"        | "online"      | ""    | FALSE     |
+------------------------------------------------------------------------------------------------+

9 rows available after 4 ms, consumed after another 1 ms
```

### Switch a database (Enterprise Edition)

```
neo4j@system> :use sales
neo4j@sales>
```

### Create or replace a database
```
neo4j@sales> match (n) return count(n) as countNode;
```
```
+-----------+
| countNode |
+-----------+
| 115       |
+-----------+

1 row available after 12 ms, consumed after another 0 ms
```

```
neo4j@system> CREATE OR REPLACE DATABASE sales;
```
```
0 rows available after 64 ms, consumed after another 0 ms
```

```
neo4j@system> SHOW DATABASES;
```

In standalone mode:
```
+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
| "sales"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

3 rows available after 2 ms, consumed after another 2 ms
```

Or in a Causal Cluster:
```
+------------------------------------------------------------------------------------------------+
| name     | address          | role       | requestedStatus | currentStatus | error | default   |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "leader"   | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7688" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7689" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "system" | "localhost:7687" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7688" | "leader"   | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7689" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7687" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7688" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7689" | "leader"   | "online"        | "online"      | ""    | FALSE     |
+------------------------------------------------------------------------------------------------+

9 rows available after 2 ms, consumed after another 2 ms
```

```
neo4j@system> :use sales
neo4j@sales> match (n) return count(n) as countNode;
```
```
+-----------+
| countNode |
+-----------+
| 0         |
+-----------+

1 row available after 15 ms, consumed after another 1 ms
```

### Stop a database
```
neo4j@system> STOP DATABASE sales;
```
```
0 rows available after 18 ms, consumed after another 6 ms
```

```
neo4j@system> SHOW DATABASES;
```
In standalone mode:
```
+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
| "sales"  | "localhost:7687" | "standalone" | "offline"       | "offline"     | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

3 rows available after 2 ms, consumed after another 1 ms
```

Or in a Causal Cluster:
```
+------------------------------------------------------------------------------------------------+
| name     | address          | role       | requestedStatus | currentStatus | error | default   |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "leader"   | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7688" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7689" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "system" | "localhost:7687" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7688" | "leader"   | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7689" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7687" | "unknown"  | "offline"       | "offline"     | ""    | FALSE     |
| "sales"  | "localhost:7688" | "unknown"  | "offline"       | "offline"     | ""    | FALSE     |
| "sales"  | "localhost:7689" | "unknown"  | "offline"       | "offline"     | ""    | FALSE     |
+------------------------------------------------------------------------------------------------+

9 rows available after 2 ms, consumed after another 1 ms
```

```
neo4j@system> :use sales
```

```
Unable to get a routing table for database 'sales' because this database is unavailable
neo4j@sales[UNAVAILABLE]>
```

### Start a database

```
neo4j@sales[UNAVAILABLE]> :use system
neo4j@system> START DATABASE sales;
0 rows available after 21 ms, consumed after another 1 ms
```

```
neo4j@system> SHOW DATABASES;
```
In standalone mode:
```
+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
| "sales"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

3 rows available after 2 ms, consumed after another 1 ms
```

Or in a Causal Cluster:
```
+------------------------------------------------------------------------------------------------+
| name     | address          | role       | requestedStatus | currentStatus | error | default   |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "leader"   | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7688" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "neo4j"  | "localhost:7689" | "follower" | "online"        | "online"      | ""    | TRUE      |
| "system" | "localhost:7687" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7688" | "leader"   | "online"        | "online"      | ""    | FALSE     |
| "system" | "localhost:7689" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7687" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7688" | "follower" | "online"        | "online"      | ""    | FALSE     |
| "sales"  | "localhost:7689" | "leader"   | "online"        | "online"      | ""    | FALSE     |
+------------------------------------------------------------------------------------------------+

9 rows available after 2 ms, consumed after another 1 ms
```

### Drop or remove a database (Enterprise Edition)
```
neo4j@system> DROP DATABASE sales;
0 rows available after 82 ms, consumed after another 1 ms
```

```
neo4j@system> SHOW DATABASES;
```
```
+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

2 rows available after 6 ms, consumed after another 0 ms
```

<a id="online_offline_backup"></a>
## Online and offline backups

Online backups are typically required for production environments, but it is also possible to perform offline backups.

Offline backups are a more limited method for backing up a database. For example:

- Online backups run against a database in online mode on a live Neo4j instance, while offline backups require that the database is shut down and Neo4j could be either live or shutdown.
- Online backups can be full or incremental, but there is no support for backing up incrementally with offline backups.

For more details about offline backups, see [Dump and load databases](#dump_and_load_databases).

The remainder of this chapter is dedicated to describing online backups.

### Configuration parameters

Configuration parameters are defined in the ```neo4j.conf``` file.

The following configuration parameters are applicable for managing databases:

| Parameter name     | Description        |       Default value                      |
|:-----------:|:---------------:|:--------------------------------------------------:|
| dbms.default_database  | Name of the default database for the Neo4j instance. The database is created if it does not exist when the instance starts. | neo4j                                      |
| dbms.max_databases    | Maximum number of databases that can be used in a Neo4j single instance or Causal Cluster. The number includes all the online and offline databases. The value is an integer with a minimum value of 2.Note that once the limit has been reached, it is not possible to create any additional databases. Similarly, if the limit is changed to a number lower than the total number of existing databases, no additional databases can be created. | 100                                   |

In a clustered setup, the value of dbms.default_database is only used to set the initial default database.

To change the default database at a later point, use the dbms.cluster.setDefaultDatabase(databaseName) procedure. This procedure is only available when running a cluster.

### Backup command
A Neo4j database can be backed up in online mode using the backup command of neo4j-admin. The machine that runs the backup command must have Neo4j installed, but does not need to run a Neo4j server. Note that it is not recommended to use an NFS mount for backup purposes as this is likely to corrupt and slow down the backup.

```
neo4j-admin backup --backup-dir=<path>
                   [--verbose=<true/false>]
                   [--from=<host:port>]
                   [--database=<database>]
                   [--fallback-to-full=<true/false>]
                   [--pagecache=<size>]
                   [--check-consistency=<true/false>]
                   [--check-graph=<true/false>]
                   [--check-indexes=<true/false>]
                   [--check-label-scan-store=<true/false>]
                   [--check-property-owners=<true/false>]
                   [--report-dir=<path>]
                   [--additional-config=<path>]
                   [--include-metadata=<all/users/roles>]
```

- Options

|     Option               |    Default      |   Description               |
|:------------------------:|:---------------:|:---------------------------:|
| --backup-dir             |                 |Directory to place backup in.|
| --verbose                |  false          |Enable verbose output.       |
| --from                   |  localhost:6362 |Host and port of Neo4j.      |
| --database               |  neo4j          |Name of the database to back up. Name can contain * or ? for name globbing. If a backup of the specified database exists in the target directory, then an incremental backup will be attempted.|
| --fallback-to-full       |  true           |If an incremental backup fails backup will move the old backup to <name>.err.<N> and fallback to a full backup instead.|
| --pagecache              |  8M             |The size of the page cache to use for the backup process.|
| --check-consistency      |  true           |If a consistency check should be made.|
| --check-graph            |  true           |Perform checks between nodes, relationships, properties, types and tokens.|
| --check-indexes          |  true           |Perform checks on indexes.|
| --check-label-scan-store |  true           |Perform checks on the label scan store.|
| --check-property-owners  |  false          |Perform additional checks on property ownership. This check is very expensive in time and memory.|
| --report-dir             |  .              |Directory where consistency report will be written.|
| --additional-config      |                 |Configuration file to supply additional configuration in.|
| --include-metadata       |                 |Include metadata in the backup. Metadata contains security settings related to the database. Can’t be used for backing system database. ‧ roles - commands to create the roles and privileges (for both database and graph) that affect the use of the database. ‧    users - commands to create the users that can use the database and their role assignments. ‧    all - include roles and users.|

The backup client can operate in two slightly different modes referred to as performing a full backup or an incremental backup. A full backup is always required initially for the very first backup into a target location. Subsequent backups will attempt to use the incremental mode where just the delta of the transcation logs since the last backup are transferred and applied onto the target location. If the required transaction logs aren’t available on the backup server then the backup client will fallback to performing a full backup instead, unless --fallback-to-full is disabled.

After the backup has been successfully performed the consistency checker will be invoked by default. Checking the consistency of the backup is a major operation which can consume significant computational resources, e.g. memory, CPU, I/O.

---
**NOTE**

It is strongly discouraged to run the backup client on a live Neo4j server, especially together with a consistency check. Doing so can adversely affect the server

---

To avoid adversely affecting a running server with the resource demands of the backup client it is recommended to take the backup and perform the consistency check on a dedicated machine which has sufficient free resources to perform the consistency check. Another alternative is to decouple the backup operation from the consistency checking and schedule that part of the workflow to happen at a later point in time on a dedicated machine. The value of consistency checking a backup should not be underestimated as it is vital for safe guarding and ensuring the quality of your data.

The transaction log files in the backup are rotated and pruned based on the provided configuration. For example, setting 
```
dbms.tx_log.rotation.retention_policy=3
```
files will keep 3 transaction log files in the backup. You can use the `--additional-config` parameter to override this configuration.

### Full backup
In this practice, set environment variables in order to control memory usage.

The page cache is defined by using the command line option `--pagecache`. Further, the HEAP_SIZE environment variable will specify the `maximum heap size` allocated to the backup process.

Now you can perform a full backup:
```
$neo4j-home> export HEAP_SIZE=2G
$neo4j-home> mkdir /mnt/backups
$neo4j-home> bin/neo4j-admin backup --from=192.168.1.34 --backup-dir=/mnt/backups/neo4j --database=neo4j --pagecache=4G
Doing full backup...
2021-02-01 14:09:09.510+0000 INFO  [o.n.c.s.StoreCopyClient] Copying neostore.nodestore.db.labels
2021-02-01 14:09:09.537+0000 INFO  [o.n.c.s.StoreCopyClient] Copied neostore.nodestore.db.labels 8.00 kB
2021-02-01 14:09:09.538+0000 INFO  [o.n.c.s.StoreCopyClient] Copying neostore.nodestore.db
2021-02-01 14:09:09.540+0000 INFO  [o.n.c.s.StoreCopyClient] Copied neostore.nodestore.db 16.00 kB
```

### Incremental backup
This practice assumes that you have performed a full backup as per the previous example. In the same way as before, make sure to control the memory usage.

To perform an incremental backup you need to specify the location of your previous backup:
```
$neo4j-home> export HEAP_SIZE=2G
$neo4j-home> bin/neo4j-admin backup --from=192.168.1.34 --backup-dir=/mnt/backups/neo4j --database=neo4j --pagecache=4G
Destination is not empty, doing incremental backup...
Backup complete.
```

### Memory configuration

- Configure heap size for the backup
This is done by setting the environment variable `HEAP_SIZE` before starting the backup program. If not specified by `HEAP_SIZE`, the Java Virtual Machine will choose a value based on server resources. `HEAP_SIZE` configures the maximum heap size allocated for the backup process.

- Configure page cache for the backup
The page cache size can be determined for the backup program by using the `--pagecache` option to the `neo4j-admin` backup command. If not explicitly defined, the page cache will default to 8MB.

<a id="dump_and_load_databases"></a>
## Dump and load databases (Offline)

A Neo4j database can be dumped and loaded using the following commands:

```
neo4j-admin dump --database=<database> --to=<destination-path>
```
```
neo4j-admin load --from=<archive-path> --database=<database> [--force]
```

### Limitations

The database must be `stopped` before running the `dump` and `load` commands.

neo4j-admin must be invoked as the neo4j user to ensure the appropriate file permissions.

If the database used to load into doesn’t exist prior to loading, it must be created subsequently using CREATE DATABASE. See * [Administrative commands](#administrative_commands) for more information.

### Server configuration

The table below lists the basic server parameters relevant to backups. Note that by default the backup service is enabled but only listens on localhost (127.0.0.1) and this needs to be changed if backups are to be taken from another machine.

| Parameter name             |         Default value            | Description                                   |
|:--------------------------:|:--------------------------------:|:---------------------------------------------:|
| dbms.backup.enabled        |          true                    | Enable support for running online backups.    |
| dbms.backup.listen_address |     127.0.0.1:6362               | Listening server for online backups.          |

<img src="images/config_setting_online_backup.png">

### Databases to backup

Since a Neo4j DBMS can host multiple databases and they are backed up independently of one another, it is important to plan a backup strategy for every database and to not forget any databases. In a new deployment there are two databases by default, neo4j and system. The system database contains configuration, e.g. operational states of databases, security configuration, etc.

### Storage considerations

For any backup it is important that you store your data separately from the production system, where there are no common dependencies, and preferably off-site. If you are running Neo4j in the cloud, you could for example use a different availability zone or even a separate cloud provider.

Since backups are kept for a long time, the longevity of archival storage should be considered as part of backup planning.

You may also want to override the settings used for pruning and rotation of transaction log files. The transaction log files are files that keep track of recent changes. Please note that removing transaction logs manually can result in a broken backup.

Recovered servers do not need all of the transaction log files that have already been applied, so it is possible to reduce storage size even further by reducing the size of the files to the bare minimum.

This can be done by setting dbms.tx_log.rotation.size=1M and dbms.tx_log.rotation.retention_policy=3 files. Alternatively you can use the --additional-config override.

### Create backup folder
```
mkdir backup
```
<img src="images/create_backup_folder.png">

### Dump database to folder
```
neo4j-admin dump --database=neo4j --to=./backup
```
<img src="images/dump_database_to_folder.png">

- Error!
```
The database is in use. Stop database 'neo4j' and try again.
```
<img src="images/dump_database_error_when_online.png">

### Stop database and dump again
```
neo4j-admin dump --database=neo4j --to=./backup
```
<img src="images/dump_database_successfully.png">

### Check backup file
<img src="images/dump_database_file.png">

### Load Database from dump file (Offline)

```
.\bin\neo4j-admin load --from=./backup/neo4j.dump --database=neo4j --force
```
<img src="images/neo4j_load_from_backup.png">

<a id="memory_heap_size"></a>
## Understanding memory configurations for neo4j-admin backup

When using `bin\neo4j-admin` backup to backup a Neo4j database, Neo4j Support recommends explicitly defining the JVM heap size and pagecache memory to be used by the backup JVM process. If these are not defined then when neo4j-admin backup is executed, the JVM will allocate a default heap size and the pagecache will use the value defined in the `dbms.memory.pagecache.size` setting from `conf/neo4j.conf`. In most cases the `conf/neo4j.conf` defines these parameters such that they consume over 60% of total RAM. As such, if neo4j-admin backup were to allocate the same values for heap and pagecache as the running Neo4j process, we would consume all available RAM leading to backup failure and out of memory conditions.

To specify the JVM heap for neo4j-admin backup, set the environment variable `HEAP_SIZE`, for example:

```
export HEAP_SIZE=2G
```

To specify the pagecache allocation for neo4j-admin backup,

- Neo4j 3.3.1

forward As a result of `PR 10463`, specify the pagecache value on the command line through `--pagecache=XXM` replacing `XX` with the about of megabytes to allocate for the pagecache. The --pagecache command line argument overrides any and all other defined pagecache settings in any `neo4j.conf` files.

- Neo4j 3.2.x through Neo4j 3.3.0

first create a directory specific for the backup config file and then set the environment variable `NEO4J_CONF`:

```
export NEO4J_CONF=/tmp/backup/
```

Then, in /tmp/backup create a file named `neo4j.conf` with the following content:
```
dbms.memory.pagecache.size=12G
```

For environments that were installed through Debian (i.e. apt-get install neo4j-enterprise) you will also need to add

```
dbms.directories.lib=/usr/share/neo4j/lib
```

Be sure not to exceed the **total physical RAM** on the server between the Neo4j database plus the backup process.


<a id="move_db_location"></a>
## Moving Neo4j Desktop database location

```
mklink /j C:\Users\<USER>\AppData\Local\Neo4j\Relate\Data\ D:\Neo4jDB\
```